In [1]:
import pandas as pd
import lifetimes
import lifetimes.datasets as ld

In [2]:
donations_df = ld.load_donations()

In [3]:
donations_df

,frequency,recency,periods,weights
0,0,0,6,3464
1,1,1,6,1091
2,1,2,6,277
3,1,3,6,129
4,1,4,6,78
5,1,5,6,119
6,1,6,6,129
7,2,2,6,613
8,2,3,6,255
9,2,4,6,155


Fit BG/BB Model

In [4]:
bgbb_mdl = lifetimes.BetaGeoBetaBinomFitter()
bgbb_mdl.fit(frequency=donations_df['frequency'],
             recency=donations_df['recency'],
             n_periods=donations_df['periods'],
             weights=donations_df['weights'])

<lifetimes.BetaGeoBetaBinomFitter: fitted with 22 subjects, alpha: 1.20, beta: 0.75, delta: 2.78, gamma: 0.66>

Predict probability customers will be after 3 months

In [5]:
donations_df['is_alive_3_months'] = bgbb_mdl.conditional_probability_alive(m_periods_in_future=3,
                                       frequency=donations_df['frequency'],
                                       recency=donations_df['recency'],
                                       n_periods=donations_df['periods'])

In [6]:
donations_df

,frequency,recency,periods,weights,is_alive_3_months
0,0,0,6,3464,0.095514
1,1,1,6,1091,0.061356
2,1,2,6,277,0.224986
3,1,3,6,129,0.425183
4,1,4,6,78,0.600985
5,1,5,6,119,0.731502
6,1,6,6,129,0.821855
7,2,2,6,613,0.058664
8,2,3,6,255,0.264023
9,2,4,6,155,0.520846


What percentage of the customer base is still alive?

In [7]:
donations_df[donations_df['is_alive_3_months'] >= 0.7]['weights'].sum() / donations_df['weights'].sum()

0.3111491354466859

Predict the number of donations in 3 months time

In [8]:
donations_df['num_tx_3_months'] = bgbb_mdl.conditional_expected_number_of_purchases_up_to_time(m_periods_in_future=3,
                                                                                               frequency=donations_df['frequency'],
                                                                                               recency=donations_df['recency'],
                                                                                               n_periods=donations_df['periods'])

In [9]:
donations_df

,frequency,recency,periods,weights,is_alive_3_months,num_tx_3_months
0,0,0,6,3464,0.095514,0.046151
1,1,1,6,1091,0.061356,0.054279
2,1,2,6,277,0.224986,0.199035
3,1,3,6,129,0.425183,0.376141
4,1,4,6,78,0.600985,0.531665
5,1,5,6,119,0.731502,0.647128
6,1,6,6,129,0.821855,0.727059
7,2,2,6,613,0.058664,0.075450
8,2,3,6,255,0.264023,0.339567
9,2,4,6,155,0.520846,0.669875


How many donations can we expect to receive in 3 months, from the customers who we predict to still be active?

In [10]:
donations_df[donations_df['is_alive_3_months'] >= 0.7]['num_tx_3_months'].mean()

1.3590980018444863